In [52]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [53]:
sc = SparkContext() # dòng này phải nằm trước dòng dưới
spark = SparkSession.builder.appName("Python Spark SQL basic example") \
    .config("spark.driver.memory", "10g") \
    .getOrCreate()

In [51]:
spark.stop()

In [3]:
# Setting the db properties
database_properties = {
    "user": "postgres",
    "password": "1",
    "driver": "org.postgresql.Driver"
}

db_name = "movie"
url_connect = f"jdbc:postgresql://localhost:5432/movie"

# Test Postgresql connection

In [ ]:
df = spark.read.format("jdbc") \
    .option("url", url_connect) \
    .option("dbtable", f'contract_duration') \
    .options(**database_properties).load()

In [5]:
df.show()

+--------+----------+-------------+-------------+-------------+-------------+
|Contract|TVDuration|MovieDuration|ChildDuration|SportDuration|RelaxDuration|
+--------+----------+-------------+-------------+-------------+-------------+
+--------+----------+-------------+-------------+-------------+-------------+



# Compute metrics

In [54]:
df_appname_category = spark.read.csv('temp/appname_category.csv', header=True)
df_appname_category.show()

+----------+---------+----------+------------+-------------+
|      Date| Contract|  Category|         Mac|TotalDuration|
+----------+---------+----------+------------+-------------+
|2022-04-01|HNH579912|TVDuration|0C96E62FC55C|          254|
|2022-04-01|HUFD40665|TVDuration|CCEDDC333614|         1457|
|2022-04-01|HNH572635|TVDuration|B068E6A1C5F6|         2318|
|2022-04-01|HND141717|TVDuration|08674EE8D2C2|         1452|
|2022-04-01|HNH743103|TVDuration|402343C25D7D|          251|
|2022-04-01|HNH893773|TVDuration|B84DEE76D3B8|          924|
|2022-04-01|HND083642|TVDuration|B84DEE849A0F|         1444|
|2022-04-01|DNFD74404|TVDuration|90324BB44C39|          691|
|2022-04-01|DTFD21200|TVDuration|B84DEED27709|         1436|
|2022-04-01|LDFD05747|TVDuration|0C96E6C95E53|         1434|
|2022-04-01|HNH063566|TVDuration|B84DEEDD1C85|          687|
|2022-04-01|HNH866786|TVDuration|10394E2790A5|          248|
|2022-04-01|NBAAA1128|TVDuration|10394E47C1AF|          247|
|2022-04-01|HNH960439|TV

In [103]:
spark

In [105]:
df = df_appname_category
date = df.select(date_format(col('Date'), 'yyyyMMdd').alias('Date')).first()['Date']
view_name = f'source_{date}'
df.createOrReplaceTempView(view_name) # cols: ["Date", "Contract", "Category", "Mac", "TotalDuration"]

df_metrics = spark.sql(f"""
    select Contract, 
        collect_set(regexp_replace(Category, 'Duration', '')) as CustomerTaste,
        count(distinct(Date)) as Activeness
    from source_{date}
    group by Contract
    having rank() over (partition by Contract order by TotalDuration desc) = 1
""")

spark.catalog.dropTempView(view_name)


AnalysisException: It is not allowed to use window functions inside HAVING clause.

In [102]:
df = df_appname_category
date = df.select(date_format(col('Date'), 'yyyyMMdd').alias('Date')).first()['Date']
view_name = f'source_{date}'
df.createOrReplaceTempView(view_name) # cols: ["Date", "Contract", "Category", "Mac", "TotalDuration"]

# df_taste = spark.sql(f"""
#     select Contract, collect_set(regexp_replace(Category, 'Duration', '')) as CustomerTaste
#     from source_{date}
#     group by Contract
# """)

# df_taste.show()

df_metrics = spark.sql(f"""
    WITH cte_most_watch as (
        select Contract, regexp_replace(Category, 'Duration', '') as MostWatch
        from (
            select *, 
                rank() over (partition by Contract order by TotalDuration desc) as rnk
            from source_{date}
        ) t
        where rnk = 1
    ),
    cte_customer_taste as (
        select Contract, 
            collect_set(regexp_replace(Category, 'Duration', '')) as CustomerTaste,
            count(distinct(Date)) as Activeness
        from source_{date}
        group by Contract
    )
    
    select w.Contract, MostWatch, CustomerTaste, t.Activeness
    from cte_most_watch w FULL OUTER JOIN cte_customer_taste t on w.Contract = t.Contract
""")

df_metrics.show()

spark.catalog.dropTempView(view_name)


+--------------+---------+-------------+----------+
|      Contract|MostWatch|CustomerTaste|Activeness|
+--------------+---------+-------------+----------+
|113.182.209.48|    Relax|  [TV, Relax]|         1|
|     AGAAA0338|       TV|         [TV]|         2|
|     AGAAA0342|       TV|         [TV]|         2|
|     AGAAA0391|       TV|         [TV]|         1|
|     AGAAA0613|       TV|         [TV]|         2|
|     AGAAA0638|       TV|         [TV]|         2|
|     AGAAA0663|       TV|         [TV]|         2|
|     AGAAA0693|       TV|  [Child, TV]|         2|
|     AGAAA0718|       TV|         [TV]|         2|
|     AGAAA0723|       TV|         [TV]|         2|
|     AGAAA0732|       TV|         [TV]|         2|
|     AGAAA0750|       TV|         [TV]|         2|
|     AGAAA0848|       TV|         [TV]|         1|
|     AGAAA0849|       TV|         [TV]|         2|
|     AGAAA0885|       TV|         [TV]|         1|
|     AGAAA0894|       TV|         [TV]|         2|
|     AGAAA0

True

In [95]:
df_metrics.orderBy(col("Activeness").desc()).show()

+---------+---------+------------------+----------+
| Contract|MostWatch|     CustomerTaste|Activeness|
+---------+---------+------------------+----------+
|AGAAA1297|    Relax|       [TV, Relax]|         2|
|AGAAA2263|       TV|              [TV]|         2|
|AGAAA1332|       TV|       [Movie, TV]|         2|
|AGD013220|       TV|              [TV]|         2|
|AGAAA1149|       TV|              [TV]|         2|
|AGAAA2273|       TV|              [TV]|         2|
|AGAAA2651|       TV|              [TV]|         2|
|AGAAA2434|    Movie|[Movie, Child, TV]|         2|
|AGAAA1139|       TV|              [TV]|         2|
|AGAAA2704|       TV|              [TV]|         2|
|AGAAA1348|       TV|              [TV]|         2|
|AGAAA0951|       TV|              [TV]|         2|
|AGAAA2143|       TV|              [TV]|         2|
|AGAAA2779|       TV|              [TV]|         2|
|AGAAA0353|       TV|       [Movie, TV]|         2|
|AGAAA1185|       TV|              [TV]|         2|
|AGAAA1267| 

# Test ETL 2 days

In [4]:
import os
parent_folder = 'E:/Dataset/log_content'
os.path.exists(parent_folder)

True

In [49]:
parent_folder = 'E:/Dataset/log_content'
list_json_files = [f"{parent_folder}/20220401.json", f"{parent_folder}/20220402.json"]

# This pattern matches the last occurrence of a forward slash (/) followed by one or more characters that are not a forward slash (the file name)
df_input = spark.read.json(list_json_files) \
    .withColumn('FileName', regexp_extract(input_file_name(), r"/([^/]+)\.json$", 1)) \
    .withColumn('Date', to_date(col('FileName'), 'yyyyMMdd')) \



In [50]:
df_input.select('Date').distinct().show(truncate=False)

+----------+
|Date      |
+----------+
|2022-04-01|
|2022-04-02|
+----------+



# Test ETL 1 day

In [8]:
df_input = spark.read.json('.\\20220401.json')
df_input.show()

+--------------------+-------+------+--------------------+-----+
|                 _id| _index|_score|             _source|_type|
+--------------------+-------+------+--------------------+-----+
|AX_momhia1FFivsGrn9o|history|     0|{KPLUS, HNH579912...|kplus|
|AX_momhca1FFivsGrnvg|history|     0|{KPLUS, HUFD40665...|kplus|
|AX_momhaa1FFivsGrnny|history|     0|{KPLUS, HNH572635...|kplus|
|AX_momhca1FFivsGrnvv|history|     0|{KPLUS, HND141717...|kplus|
|AX_momhia1FFivsGrn98|history|     0|{KPLUS, HNH743103...|kplus|
|AX_momg9a1FFivsGrnkS|history|     0|{KPLUS, HNH893773...|kplus|
|AX_momhca1FFivsGrnwA|history|     0|{KPLUS, HND083642...|kplus|
|AX_momhfa1FFivsGrn2u|history|     0|{KPLUS, DNFD74404...|kplus|
|AX_momhca1FFivsGrnwP|history|     0|{KPLUS, DTFD21200...|kplus|
|AX_momhca1FFivsGrnwU|history|     0|{KPLUS, LDFD05747...|kplus|
|AX_momhfa1FFivsGrn24|history|     0|{KPLUS, HNH063566...|kplus|
|AX_momhia1FFivsGrn-W|history|     0|{KPLUS, HNH866786...|kplus|
|AX_momhia1FFivsGrn-a|his

In [11]:
df_input.select("_type").distinct().show()

+-------+
|  _type|
+-------+
|  kplus|
|    vod|
|  relax|
|channel|
|  child|
|  sport|
|   fims|
+-------+



In [5]:
df_input.printSchema()

root
 |-- _id: string (nullable = true)
 |-- _index: string (nullable = true)
 |-- _score: long (nullable = true)
 |-- _source: struct (nullable = true)
 |    |-- AppName: string (nullable = true)
 |    |-- Contract: string (nullable = true)
 |    |-- Mac: string (nullable = true)
 |    |-- TotalDuration: long (nullable = true)
 |-- _type: string (nullable = true)



In [19]:
df_source = df_input.select( '_source.Contract', '_source.AppName', '_source.TotalDuration', '_source.Mac')
df_source.show()

+---------+-------+-------------+------------+
| Contract|AppName|TotalDuration|         Mac|
+---------+-------+-------------+------------+
|HNH579912|  KPLUS|          254|0C96E62FC55C|
|HUFD40665|  KPLUS|         1457|CCEDDC333614|
|HNH572635|  KPLUS|         2318|B068E6A1C5F6|
|HND141717|  KPLUS|         1452|08674EE8D2C2|
|HNH743103|  KPLUS|          251|402343C25D7D|
|HNH893773|  KPLUS|          924|B84DEE76D3B8|
|HND083642|  KPLUS|         1444|B84DEE849A0F|
|DNFD74404|  KPLUS|          691|90324BB44C39|
|DTFD21200|  KPLUS|         1436|B84DEED27709|
|LDFD05747|  KPLUS|         1434|0C96E6C95E53|
|HNH063566|  KPLUS|          687|B84DEEDD1C85|
|HNH866786|  KPLUS|          248|10394E2790A5|
|NBAAA1128|  KPLUS|          247|10394E47C1AF|
|HNH960439|  KPLUS|          683|B84DEED34371|
|HNJ035736|  KPLUS|          246|CCD4A1FA86A5|
|NTFD93673|  KPLUS|         2288|B84DEEEF4763|
|HNJ063267|  KPLUS|         2282|10394E172CA7|
|HNH790383|  KPLUS|          906|4CEBBD53378B|
|THFD12466|  

In [17]:
df_source.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- AppName: string (nullable = true)
 |-- TotalDuration: long (nullable = true)



In [25]:
# rule
state_dict = {
    'CHANNEL':'Truyền Hình',
    'DSHD':'Truyền Hình', 
    'KPLUS':'Truyền Hình',
    'VOD' : 'Phim truyện', 
    'FIMS': 'Phim truyện',
    'SPORT': 'Thể thao', 
    'RELAX': 'Giải trí',
    'CHILD': 'Thiếu nhi'  
}

In [26]:
df_category = df_source.rdd \
    .map(lambda row: (row.Contract, state_dict[row.AppName], row.Mac, row.TotalDuration)) \
    .toDF(["Contract", "Category", "Mac", "TotalDuration"])
    
df_category.show()

+---------+-----------+------------+-------------+
| Contract|   Category|         Mac|TotalDuration|
+---------+-----------+------------+-------------+
|HNH579912|Truyền Hình|0C96E62FC55C|          254|
|HUFD40665|Truyền Hình|CCEDDC333614|         1457|
|HNH572635|Truyền Hình|B068E6A1C5F6|         2318|
|HND141717|Truyền Hình|08674EE8D2C2|         1452|
|HNH743103|Truyền Hình|402343C25D7D|          251|
|HNH893773|Truyền Hình|B84DEE76D3B8|          924|
|HND083642|Truyền Hình|B84DEE849A0F|         1444|
|DNFD74404|Truyền Hình|90324BB44C39|          691|
|DTFD21200|Truyền Hình|B84DEED27709|         1436|
|LDFD05747|Truyền Hình|0C96E6C95E53|         1434|
|HNH063566|Truyền Hình|B84DEEDD1C85|          687|
|HNH866786|Truyền Hình|10394E2790A5|          248|
|NBAAA1128|Truyền Hình|10394E47C1AF|          247|
|HNH960439|Truyền Hình|B84DEED34371|          683|
|HNJ035736|Truyền Hình|CCD4A1FA86A5|          246|
|NTFD93673|Truyền Hình|B84DEEEF4763|         2288|
|HNJ063267|Truyền Hình|10394E17

In [29]:
df_duration_by_category = df_category \
    .groupBy("Contract", "Category") \
    .agg(sum("TotalDuration").alias("TotalDuration")) \
    .orderBy("Contract")
    
df_duration_by_category.show()

+--------------+-----------+-------------+
|      Contract|   Category|TotalDuration|
+--------------+-----------+-------------+
|             0|Truyền Hình|    519546866|
|113.182.209.48|Truyền Hình|           63|
|113.182.209.48|   Giải trí|           89|
|     AGAAA0335|Truyền Hình|        11440|
|     AGAAA0338|Truyền Hình|         8895|
|     AGAAA0342|Truyền Hình|           44|
|     AGAAA0345|Truyền Hình|        13846|
|     AGAAA0346|Truyền Hình|        47203|
|     AGAAA0350|Truyền Hình|        11090|
|     AGAAA0352|Truyền Hình|        10094|
|     AGAAA0353|Truyền Hình|           16|
|     AGAAA0356|Truyền Hình|        39995|
|     AGAAA0357|Truyền Hình|           61|
|     AGAAA0359|Truyền Hình|        86400|
|     AGAAA0360|Truyền Hình|         6188|
|     AGAAA0366|Truyền Hình|        51934|
|     AGAAA0370|Truyền Hình|        85932|
|     AGAAA0372|Truyền Hình|           19|
|     AGAAA0375|Phim truyện|          114|
|     AGAAA0375|Truyền Hình|        13545|
+----------

=> Mỗi contract có thể thuộc về 2 category, dẫn tới có thể có 2 dòng trùng cho 1 contract -> Pivot

In [30]:
df_pivot = df_duration_by_category.groupBy("Contract").pivot("Category").agg(sum("TotalDuration").alias("TotalDuration"))
df_pivot.show()

+---------+--------+-----------+---------+--------+-----------+
| Contract|Giải trí|Phim truyện|Thiếu nhi|Thể thao|Truyền Hình|
+---------+--------+-----------+---------+--------+-----------+
|HPFD48556|      69|       NULL|     NULL|    NULL|      92976|
|NBFD10014|    NULL|       NULL|     NULL|    NULL|      84628|
|HNH036174|    NULL|       NULL|     NULL|    NULL|       6049|
|DNH067877|    NULL|       NULL|     NULL|    NULL|       5760|
|SGH806190|    NULL|       NULL|     NULL|    NULL|       1131|
|HDFD42710|    NULL|       NULL|     NULL|    NULL|      12096|
|NTFD85431|    NULL|       2759|       39|    NULL|        974|
|NTFD24068|    NULL|       3836|     NULL|    NULL|        281|
|DAD046926|    NULL|       1700|     3315|    NULL|      14976|
|HUFD61701|    NULL|       1983|     NULL|    NULL|        445|
|NDFD32943|    NULL|       NULL|     NULL|    NULL|       6269|
|TNFD30439|    NULL|       NULL|     NULL|    NULL|       5238|
|DNH083321|    NULL|       NULL|     NUL

In [32]:
df_pivot.withColumn("Date", to_date(lit("20240114"), "yyyyMMdd")).show()

+---------+--------+-----------+---------+--------+-----------+----------+
| Contract|Giải trí|Phim truyện|Thiếu nhi|Thể thao|Truyền Hình|      Date|
+---------+--------+-----------+---------+--------+-----------+----------+
|HPFD48556|      69|       NULL|     NULL|    NULL|      92976|2024-01-14|
|NBFD10014|    NULL|       NULL|     NULL|    NULL|      84628|2024-01-14|
|HNH036174|    NULL|       NULL|     NULL|    NULL|       6049|2024-01-14|
|DNH067877|    NULL|       NULL|     NULL|    NULL|       5760|2024-01-14|
|SGH806190|    NULL|       NULL|     NULL|    NULL|       1131|2024-01-14|
|HDFD42710|    NULL|       NULL|     NULL|    NULL|      12096|2024-01-14|
|NTFD85431|    NULL|       2759|       39|    NULL|        974|2024-01-14|
|NTFD24068|    NULL|       3836|     NULL|    NULL|        281|2024-01-14|
|DAD046926|    NULL|       1700|     3315|    NULL|      14976|2024-01-14|
|HUFD61701|    NULL|       1983|     NULL|    NULL|        445|2024-01-14|
|NDFD32943|    NULL|     

In [38]:
import os

folder_month_path = "./one_month/"
log_files = [folder_month_path + file_name for file_name in os.listdir(folder_month_path) if file_name.endswith(".json")]
log_files
# [folder_month_path + file_name for file_name in os.listdir(folder_month_path) if file_name.endswith(".json")]

['./one_month/20220401.json', './one_month/20220515.json']